# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

In [1]:
import csv
import sqlite3
from datetime import datetime

In [16]:
def as_int(x: str) -> int:
    if x.isdecimal():
        return int(x)


def as_float(x: str) -> float:
    if x.isdecimal():
        return float(x)


def as_timestamp(x: str, fmt='%Y-%m-%d') -> int:
    return int(datetime.strptime(x, fmt).timestamp())

In [5]:
con = sqlite3.connect('data/db/Chinook_Sqlite.sqlite')
cur = con.cursor()

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [6]:
stmt = """
select FirstName, LastName from Customer where Country == 'Canada';
"""
cur.execute(stmt)
cur.fetchall()

[('François', 'Tremblay'),
 ('Mark', 'Philips'),
 ('Jennifer', 'Peterson'),
 ('Robert', 'Brown'),
 ('Edward', 'Francis'),
 ('Martha', 'Silk'),
 ('Aaron', 'Mitchell'),
 ('Ellie', 'Sullivan')]

2. Найти и вывести на экран названия всех альбомов группы Accept

In [7]:
stmt = """
select Title from Album
where ArtistId = (select ArtistId from Artist where Name == 'Accept');
"""
cur.execute(stmt)
cur.fetchall()
con.close()

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [8]:
con = sqlite3.connect('data/db/pi19_3.sqlite')
cur = con.cursor()

In [9]:
stmt = """
create table if not exists Student (
    id integer not null primary key,
    name varchar not null
)
"""
cur.execute(stmt)

In [10]:
stmt = """
insert into Student (name) values ('Alexey Kovalev');
"""
cur.execute(stmt)
con.commit()

In [11]:
stmt = """
select * from Student
"""
cur.execute(stmt)
students = cur.fetchall()
con.close()
students

[(1, 'Alexey Kovalev'),
 (2, 'Alexey Kovalev'),
 (3, 'Alexey Kovalev'),
 (4, 'Alexey Kovalev')]

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [3]:
con = sqlite3.connect('data/db/recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [4]:
stmt = """
create table if not exists Recipe (
    id integer not null,
    name varchar,
    minutes real,
    submitted integer,
    description text,
    n_ingredients integer,
    constraint 'recipe_pk' primary key (id)
)
"""
cur.execute(stmt)

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [5]:
stmt = """
create table if not exists Review (
    id integer not null,
    user_id integer,
    recipe_id integer,
    date integer,
    rating real,
    review text,
    constraint 'review_pk' primary key (id),
    constraint 'recipe_fk' foreign key (recipe_id) references Recipe(id)
)
"""
cur.execute(stmt)
con.commit()

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [6]:
with open('data/reviews_sample.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    cast = [
        as_int,
        as_int,
        as_int,
        as_timestamp,
        as_float,
        str
    ]
    data = [[cast[i](row[i]) for i in range(len(row))] for row in reader]

In [7]:
stmt = """
insert into Review (id, user_id, recipe_id, date, rating, review) values (?, ?, ?, ?, ?, ?)
"""
cur.executemany(stmt, data)
con.commit()

In [17]:
with open('data/recipes_sample_with_filled_nsteps.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)
    index = [2, 1, 3, 5, 7, 8]
    cast = [as_int, str, as_float, as_timestamp, str, as_int]
    data = []
    for row in reader:
        data.append([cast[j](row[i]) for j, i in enumerate(index)])

ValueError: invalid literal for int() with base 10: '18.0'

In [9]:
stmt = """
insert into Recipe (id, name, minutes, submitted, description, n_ingredients) values (?, ?, ?, ?, ?, ?)
"""
cur.executemany(stmt, data)
con.commit()
con.close()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [19]:
con = sqlite3.connect('data/db/recipes.db')
cur = con.cursor()

In [20]:
stmt = """
select * from Recipe where n_ingredients == 10 limit 5;
"""
cur.execute(stmt)
cur.fetchall()

[]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [21]:
stmt = """
select * from Recipe order by minutes desc limit 1;
"""
cur.execute(stmt)
cur.fetchall()

[(236274,
  'strawberry liqueur',
  129615.0,
  1182373200,
  'a beautiful light red shade with a delicious strawberry flavor.',
  None)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [28]:

def find_recipe_by_id():
    recipe_id = input('recipe_id: ')
    if not recipe_id.isdecimal():
        print('Not id')
        return

    recipe_id = int(recipe_id)
    stmt = """
    select * from Recipe where id == ?
    """
    cur.execute(stmt, (recipe_id, ))
    res = cur.fetchall()

    if not res:
        print('Not found')
        return

    return res

In [29]:
find_recipe_by_id()

Not found


8. Найдите кол-во отзывов с рейтингом 5.

In [30]:
stmt = """
select * from Review where rating == 5;
"""
cur.execute(stmt)
cur.fetchall()

[(9, 273745, 134728, 1135198800, 5.0, 'Better than the real!!'),
 (11,
  190375,
  134728,
  1173387600,
  5.0,
  'These taste absolutely wonderful!!  My son-in-law loves them and requests them often! I followed the recipe exactly.  Thanks so much for posting this recipe.'),
 (13,
  255338,
  134728,
  1207861200,
  5.0,
  'First time using liquid smoke in a recipe. Made this as directed. My kids enjoyed this, so we will make this again. Thank you for posting.'),
 (14,
  1171894,
  134728,
  1240261200,
  5.0,
  "MMMMM! This is so good! I actually soaked the chicken in chicken bouillon first, and it made it super flavorful. I use Sweet Baby Ray's bbq sauce (DH's favorite), and cut chicken into nuggets instead of strips. This makes a good amount, so you could really freeze some and reheat in oven later. Those were our plans, but it was so good that it didn't make it to the ziploc bag!"),
 (18,
  217118,
  200236,
  1208466000,
  5.0,
  "This was a lovely Morrocan style dish. I halved th

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.